# PRS map - R2 vs h2 plot

Yosuke Tanigawa


## library, functions, and constants

In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [2]:
source('paths.sh')
devtools::load_all(cud4_d)
devtools::load_all(dirname(dirname(snpnet_helper)))
source(snpnet_helper)


ℹ Loading cud4

ℹ Loading snpnet



In [3]:
traits_w_metrics_f %>%
fread() %>%
rename_with(
    function(x){str_replace(x, '#', '')}, starts_with("#")
) -> traits_w_metrics_df


eval_fullwDelta_f %>%
fread() %>%
rename_with(
    function(x){str_replace(x, '#', '')}, starts_with("#")
) -> eval_long_df


In [6]:
# GWAS h2 table (heritability)
GWAS_h2_f %>% 
fread() %>%
rename_with(
    function(x){str_replace(x, '#', '')}, starts_with("#")
) -> GWAS_h2_df


## R2 vs h2 plot

In [4]:
p_h2_vs_geno_labels <- data.frame(
    plot_label = c(
        # Binary traits
        'Started insulin within one year diagnosis of diabetes',
        'JCV seropositivity for Human Polyomavirus JCV',
        'MCV seropositivity for Merkel Cell Polyomavirus',
        'Ease of skin tanning\n(Never tan, only burn)',
        'Hair color (red)',
        'Hair color (blonde)',
        'Hair color\n(dark brown)',
        'Hair color (brown)',
        'Hair color (black)',
        'Hair color (light brown)',
        'Hypertension',
        'High Blood Pressure diagnosed by doctor',
        # Quantitative traits
        'Platelet distribution width',
        'Mean corpuscular volume',
        'Platelet count',
        '6mm weak meridian (left)',
        'Lipoprotein A',
        'Total bilirubin',
        'Direct billirubin',
        'Mean platelet volume',
        'EBNA-1 antigen for Epstein-Barr Virus',
        'Standing height'
    ),
    trait = c(
        # Binary traits
        'BIN_FC10002986',
        'BIN23066',
        'BIN23067',
        'BIN_FC40001727',
        'BIN_FC2001747',
        'BIN_FC1001747',
        'BIN_FC4001747',
        'BIN_FC7001747',
        'BIN_FC5001747',
        'BIN_FC3001747',
        'HC215',
        'BIN_FC4006150',
        # Quantitative traits
        'INI30110',
        'INI30040',
        'INI30080',
        'INI5097',
        'INI30790',
        'INI30840',
        'INI30660',
        'INI30100',
        'INI23004',
        'INI50'
    ),
    stringsAsFactors=F
)

In [5]:
p_h2_vs_geno_sub_labels <- data.frame(
    plot_sub_label = c(
        # Binary traits
        'Asthma diagnosed by doctor',
        'Hayfever rhinitis or eczema diagnosed by doctor',
        'Hypothyroidism/myxoedema',
        'Hayfever allergic rhinitis or eczema',
        'Never Smoker',
        'TTE disorders of\nporphyrin and\nbilirubin metabolism',
        'Coeliac disease',
        'Non-melanoma skin cancer',
        'Skin cancer',
        'Diabetes',
        'Psoriasis',
#         'Snoring',
        'Worrier / anxious feelings',
#         'Past tobacco smoking\n(Smoked at least once)',
        
        # Quantitative traits
        'Apolipoprotein B',
        'Mean reticulocyte volume',
        'Heel BMD', #bone mineral density
#         'Apolipoprotein A',
        'IGF-1',
        'Red blood cell count',
#         'High light scatter\nreticulocyte count'
        'LDL cholesterol',
        'Cholesterol',
        'Neutrophill %',
        'Peak expiratory flow\npredicted ratio',
        'Vitamin D',
        'Basophill %'
    ),
    trait = c(
        # Binary traits
        'BIN_FC4006152',
        'BIN_FC5006152',
        'HC219',
        'BIN_FC10006152',
        'BIN_FC10020116',
        'HC702',
        'HC303',
        'cancer1060',
        'cancer1003',    
        'HC221',
        'HC38',
#         'BIN1210',
        'BIN1980',
#         'BIN_FC40001249',
        
        # Quantitative traits
        'INI30640',
        'INI30260',
        'INI3148',
#         'INI30630',
        'INI30770',
        'INI30010',
#         'INI30300'
        'INI30780',
        'INI30690',
        'INI30200',
        'INI1003064',
        'INI30890',
        'INI30220'
    ),
    stringsAsFactors=F
)

In [11]:
h2_vs_geno_df <- eval_long_df %>%
filter(
    WBtest_is_significant,
    model=='PRS',
    split == 'test'
) %>%
left_join(
    GWAS_h2_df %>%
    rename('trait'='p') %>%
    select(trait, h2_obs, h2_obs_se) %>%
    unique %>%
    filter(! is.na(h2_obs)),
    by="trait"
) %>%
left_join(p_h2_vs_geno_labels, by='trait') %>%
left_join(p_h2_vs_geno_sub_labels, by='trait') %>%
replace_na(list('plot_label'='', 'plot_sub_label'=''))


### linear regression (h2 vs R2)

In [13]:
h2_vs_geno_df %>%
count(family, metric)

family,metric,n
<chr>,<chr>,<int>
binomial,auc,244
binomial,NagelkerkeR2,244
binomial,TjurR2,244
gaussian,r2,569


In [15]:
h2_vs_geno_df %>%
count(family, metric) %>%
left_join(
    h2_vs_geno_df$metric %>% unique %>%
    lapply(function(metric_selected){
        glm(
            value ~ 0 + h2_obs,
            data = h2_vs_geno_df %>% filter(metric == metric_selected)
        ) %>%
        fit_to_df() %>%
        mutate(metric = metric_selected)
    }) %>% bind_rows,
    by = "metric"
) -> h2_vs_geno_regression_df

In [16]:
h2_vs_geno_regression_df


family,metric,n,variable,estimate,SE,z_or_t_value,P
<chr>,<chr>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
binomial,auc,244,h2_obs,3.6563313,0.404883231,9.030582,5.416276e-17
binomial,NagelkerkeR2,244,h2_obs,0.3239797,0.043020219,7.530870,9.866512e-13
binomial,TjurR2,244,h2_obs,0.2019018,0.029154490,6.925237,3.865663e-11
gaussian,r2,569,h2_obs,0.1521934,0.008071877,18.854775,6.041847e-62


## correlation test

In [17]:
cor_test_h2_R2_wrapper <- function(df, cor_test_method){cor.test(
    df %>% pull(h2_obs),
    df %>% pull(value),
    method = cor_test_method
)}


In [19]:
h2_vs_geno_rho <- list()

for(metric_selected in unique(h2_vs_geno_df$metric)){
    h2_vs_geno_rho[[metric_selected]] <- h2_vs_geno_df %>%
    filter(metric == metric_selected) %>%
    cor_test_h2_R2_wrapper("spearman")
}


Warning message in cor.test.default(df %>% pull(h2_obs), df %>% pull(value), method = cor_test_method):
“Cannot compute exact p-value with ties”
Warning message in cor.test.default(df %>% pull(h2_obs), df %>% pull(value), method = cor_test_method):
“Cannot compute exact p-value with ties”
Warning message in cor.test.default(df %>% pull(h2_obs), df %>% pull(value), method = cor_test_method):
“Cannot compute exact p-value with ties”
Warning message in cor.test.default(df %>% pull(h2_obs), df %>% pull(value), method = cor_test_method):
“Cannot compute exact p-value with ties”


In [134]:
# focusing on non-biomarker traits only
h2_vs_geno_rho[["r2noBiomarkers"]] <- h2_vs_geno_df %>%
filter(metric == "r2") %>%
filter(trait_category != "Biomarkers") %>%
cor_test_h2_R2_wrapper("spearman")


Warning message in cor.test.default(df %>% pull(h2_obs), df %>% pull(value), method = cor_test_method):
“Cannot compute exact p-value with ties”


In [135]:
h2_vs_geno_df %>%
count(family, metric) %>%
full_join(
    h2_vs_geno_rho %>% names() %>%
    lapply(function(metric_selected){
        h2_vs_geno_rho[[metric_selected]] %>%
        broom::tidy() %>% as.data.frame() %>%
        mutate(metric = metric_selected)
    }) %>% bind_rows(),
    by = "metric"
) -> h2_vs_geno_rho_df


In [136]:
h2_vs_geno_rho_df


family,metric,n,estimate,statistic,p.value,method,alternative
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
binomial,auc,244,0.2225175,1882355.2,4.617919e-04,Spearman's rank correlation rho,two.sided
binomial,NagelkerkeR2,244,0.4436029,1347087.5,3.470153e-13,Spearman's rank correlation rho,two.sided
binomial,TjurR2,244,0.7694916,558081.5,4.976507e-49,Spearman's rank correlation rho,two.sided
gaussian,r2,569,0.4630409,16486384.8,1.396781e-31,Spearman's rank correlation rho,two.sided
NA,r2noBiomarkers,NA,0.5432095,11658043.3,2.144479e-42,Spearman's rank correlation rho,two.sided


## plots

In [28]:
plot_h2_vs_geno <- function(h2_vs_geno_df, metric_selected){
    h2_vs_geno_df %>%
    filter(metric == metric_selected) %>%
    ggplot(aes(x  = h2_obs, y = value, color=trait_category_plot, label=trait_label)) +
    geom_errorbarh(aes(xmin = h2_obs - h2_obs_se, xmax = h2_obs + h2_obs_se), alpha=.2) +
    # geom_abline(
    #     intercept=0,
    #     slope=h2_vs_geno_regression_df %>% filter(family == 'binomial') %>% pull(estimate),
    #     color='gray',
    #     linetype = "dashed"
    # ) +
    geom_abline(intercept=0, slope=1, color='gray') +
    geom_point(alpha=.5) +
    theme_bw(base_size = 16) +
    labs(
        x = 'Estimated SNP-based heritability [SE]',
        color = 'Trait category'
    ) + 
    theme(legend.position = 'bottom') +
    guides(
      color = guide_legend(
        title = 'Trait category',
        override.aes = aes(label = "", alpha=1),
        ncol=2
      )
    )
}



In [32]:
plot_h2_vs_geno_annotate_rho <- function(plot_obj, rho_df, metric_selected, ypos = c(.48, .46)){
    plot_obj +
    annotate(
        geom="text", x = -0.02, y = ypos[1], color="black",
        hjust = 0, parse = TRUE, size = 7,
        label=sprintf(
            "\"Spearman's\" ~ rho == %0.2f",
            round(rho_df %>% filter(metric == metric_selected) %>% pull(estimate), 2)
        ),
    ) + 
    annotate(
        geom="text", x = -0.02, y = ypos[2],color="black",
        hjust = 0, parse = TRUE, size = 7,
        label = sprintf(
            "\"(p-value: \" * %.1e * \")\"",
            rho_df %>% filter(metric == metric_selected) %>% pull(p.value)
        )
    )
}


In [127]:
p_h2_vs_geno_r2 <- (
    h2_vs_geno_df %>%
    filter(family == "gaussian") %>%
    plot_h2_vs_geno("r2") +
    labs(
        title = 'Quantitative traits (Gaussian model)',
        y = latex2exp::TeX('Predictive performance (\\textit{R}$^2$)'),
    ) +
    annotation_custom(
        ggplotGrob(
            h2_vs_geno_df %>%
            filter(family == "gaussian") %>%
            plot_h2_vs_geno("r2")  +
            theme(
                legend.position = 'none',
                axis.title.x=element_blank(),
                axis.title.y=element_blank(),
                title=element_blank()
            ) +
            xlim(0, 0.3) + ylim(0, 0.15) +
            ggrepel::geom_text_repel(max.overlaps=30, force=20, mapping = aes(label = plot_sub_label))
        ), 
        xmin = .5, xmax = 1.28, ymin = .16, ymax = 0.56
    ) +
    geom_rect(mapping=aes(xmin=0, xmax=.3, ymin=0, ymax=0.15), color="black", alpha=0, size=.1)
) %>%
plot_h2_vs_geno_annotate_rho(h2_vs_geno_rho_df, "r2") +        
ggrepel::geom_text_repel(max.overlaps=30, force=20, mapping = aes(label = plot_label))

Warning message:
“Removed 172 rows containing missing values (geom_errorbarh).”
Warning message:
“Removed 113 rows containing missing values (geom_point).”
Warning message:
“Removed 113 rows containing missing values (geom_text_repel).”


In [145]:
p_h2_vs_geno_r2noBiomarkers <- (
    h2_vs_geno_df %>%
    filter(family == "gaussian") %>%
    filter(trait_category != "Biomarkers") %>%
    plot_h2_vs_geno("r2") +
    labs(
        title = 'Quantitative traits (Gaussian model)\n(non-biomarker traits only)',
        y = latex2exp::TeX('Predictive performance (\\textit{R}$^2$)'),
    ) +
    annotation_custom(
        ggplotGrob(
            h2_vs_geno_df %>%
            filter(family == "gaussian") %>%
            filter(trait_category != "Biomarkers") %>%
            plot_h2_vs_geno("r2")  +
            theme(
                legend.position = 'none',
                axis.title.x=element_blank(),
                axis.title.y=element_blank(),
                title=element_blank()
            ) +
            xlim(0, 0.3) + ylim(0, 0.15) +
            ggrepel::geom_text_repel(max.overlaps=30, force=20, mapping = aes(label = plot_sub_label))
        ), 
        xmin = .55, xmax = 1.28, ymin = .16, ymax = 0.37
    ) +
    geom_rect(mapping=aes(xmin=0, xmax=.3, ymin=0, ymax=0.15), color="black", alpha=0, size=.1)
) %>%
plot_h2_vs_geno_annotate_rho(h2_vs_geno_rho_df, "r2noBiomarkers", ypos = c(.34, .32)) +        
ggrepel::geom_text_repel(max.overlaps=30, force=20, mapping = aes(label = plot_label))

Warning message:
“Removed 167 rows containing missing values (geom_errorbarh).”
Warning message:
“Removed 108 rows containing missing values (geom_point).”
Warning message:
“Removed 108 rows containing missing values (geom_text_repel).”


In [128]:
p_h2_vs_geno_NagelkerkeR2 <- (
    h2_vs_geno_df %>%
    filter(family == "binomial") %>%
    plot_h2_vs_geno("NagelkerkeR2") +
    labs(
        title = 'Binary traits (Binomial model)',
        y = latex2exp::TeX('Predictive performance (Nagelkerke\'s pseudo \\textit{R}$^2$)'),
    ) +
    annotation_custom(
        ggplotGrob(
            h2_vs_geno_df %>%
            filter(family == "binomial") %>%
            plot_h2_vs_geno("NagelkerkeR2") +
            theme(
                legend.position = 'none',
                axis.title.x=element_blank(),
                axis.title.y=element_blank(),
                title=element_blank()
            ) +
            xlim(0, 0.118) + ylim(0, 0.075) +
            ggrepel::geom_text_repel(max.overlaps=30, force=20, mapping = aes(label = plot_sub_label))
        ), 
        xmin = .3, xmax = 1.15, ymin = .15, ymax = 0.61
    ) +
    geom_rect(mapping=aes(xmin=0, xmax=.118, ymin=0, ymax=0.075), color="black", alpha=0, size=.1)
) %>%
plot_h2_vs_geno_annotate_rho(h2_vs_geno_rho_df, "NagelkerkeR2", ypos = c(.55, .52)) +        
ggrepel::geom_text_repel(max.overlaps=30, force=20, mapping = aes(label = plot_label))

Warning message:
“Removed 21 rows containing missing values (geom_errorbarh).”
Warning message:
“Removed 20 rows containing missing values (geom_point).”
Warning message:
“Removed 20 rows containing missing values (geom_text_repel).”


In [129]:
p_h2_vs_geno_TjurR2 <- (
    h2_vs_geno_df %>%
    filter(family == "binomial") %>%
    plot_h2_vs_geno("TjurR2") +
    labs(
        title = 'Binary traits (Binomial model)',
        y = latex2exp::TeX('Predictive performance (Tjur\'s pseudo \\textit{R}$^2$)'),
    ) +
    annotation_custom(
        ggplotGrob(
            h2_vs_geno_df %>%
            filter(family == "binomial") %>%
            plot_h2_vs_geno("TjurR2") +
            theme(
                legend.position = 'none',
                axis.title.x=element_blank(),
                axis.title.y=element_blank(),
                title=element_blank()
            ) +
            xlim(0, 0.15) + ylim(0, 0.038) +
            ggrepel::geom_text_repel(max.overlaps=30, force=20, mapping = aes(label = plot_sub_label))
        ), 
        xmin = .3, xmax = 1.15, ymin = .12, ymax = 0.48
    ) +
    geom_rect(mapping=aes(xmin=0, xmax=.15, ymin=0, ymax=0.038), color="black", alpha=0, size=.1)
) %>%
plot_h2_vs_geno_annotate_rho(h2_vs_geno_rho_df, "TjurR2", ypos = c(.45, .425)) +        
ggrepel::geom_text_repel(max.overlaps=30, force=20, mapping = aes(label = plot_label))

Warning message:
“Removed 14 rows containing missing values (geom_errorbarh).”
Warning message:
“Removed 14 rows containing missing values (geom_point).”
Warning message:
“Removed 14 rows containing missing values (geom_text_repel).”


In [146]:
for(ext in c('png', 'pdf')){
    ggsave(
        file.path("plots", sprintf('h2_vs_geno.%s', ext)),
        gridExtra::arrangeGrob(
            p_h2_vs_geno_NagelkerkeR2,
            p_h2_vs_geno_r2,
            # format
            ncol=2
        ),
        width=20, height=11
    )
    ggsave(
        file.path("plots", sprintf('h2_vs_geno_NagelkerkeR2.%s', ext)),
        p_h2_vs_geno_NagelkerkeR2,
        width=10, height=11
    )
    ggsave(
        file.path("plots", sprintf('h2_vs_geno_gaussian.%s', ext)),
        p_h2_vs_geno_r2,
        width=10, height=11
    )
    ggsave(
        file.path("plots", sprintf('h2_vs_geno_gaussian_noBiomarkers.%s', ext)),
        p_h2_vs_geno_r2noBiomarkers,
        width=10, height=11
    )
    ggsave(
        file.path("plots", sprintf('h2_vs_geno_TjurR2.%s', ext)),
        p_h2_vs_geno_TjurR2,
        width=10, height=11
    )

}


In [131]:
htmlwidgets::saveWidget(
    (
        h2_vs_geno_df %>%
        filter(family == "binomial") %>%
        plot_h2_vs_geno("NagelkerkeR2") +
        labs(
            title = 'Binary traits (Binomial model)',
            y = "Predictive performance (Nagelkerke's pseudo-R^2)",
        )
    ) %>% plotly::ggplotly(),
    'ggplotly/h2_vs_geno_NagelkerkeR2.html'
)

htmlwidgets::saveWidget(
    (
        
        h2_vs_geno_df %>%
        filter(family == "gaussian") %>%
        plot_h2_vs_geno("r2") +
        labs(
            title = 'Quantitative traits (Gaussian model)',
            y = "Predictive performance (R^2)",
        )
    ) %>% plotly::ggplotly(),
    'ggplotly/h2_vs_geno_gaussian.html'
)

htmlwidgets::saveWidget(
    (
        h2_vs_geno_df %>%
        filter(family == "binomial") %>%
        plot_h2_vs_geno("TjurR2") +
        labs(
            title = 'Binary traits (Binomial model)',
            y = "Predictive performance (Tjur's pseudo-R^2)",
        )
    ) %>% plotly::ggplotly(),
    'ggplotly/h2_vs_geno_TjurR2.html'
)
